In [1]:
import networkx as nx
import numpy as np
import sys
from random import random
from random import sample
from random import choice
from random import randint
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import sklearn.metrics 
from scipy import interp

In [2]:
#Functions from file util.py

def load(fname, directed):
    import numpy as np
    a = np.loadtxt(fname, dtype=np.int)
    a[:,:2] -= 1    #convert nodes to 0-indexing
    numnodes = len(np.unique(a))
    g = np.zeros((numnodes, numnodes))
    for i in range(a.shape[0]):
        g[a[i, 0], a[i, 1]] = 1
        if not directed:
            g[a[i, 1], a[i, 0]] = 1
    return g

def strip(fname):
    import networkx as nx
    with open(fname) as f:
        lines = f.readlines()
    lines = [int(x.strip()) for x in lines]
    return lines

def strip_demo(fname):
    import networkx as nx
    with open(fname) as f:
        lines = [line.split() for line in f]

    new_list = []
    for sublist in lines:
        new_sublist = []
        for each in sublist:
            each = int(each)
            new_sublist.append(each)
        new_list.append(new_sublist)

    return new_list

def load_edgelist(fname):
    import numpy as np
    import networkx as nx
    a = np.loadtxt(fname, dtype=np.int)
    #make_ids_contiguous(a)
    g = nx.from_edgelist(a)
    return g
    
def make_ids_contiguous(a):
    #Input: the array from the original text file input
    #Output: the same array, but with the node IDs in a contiguous block starting
    #from 0. This lets the node IDs serve as array indices.

    import numpy as np
    next_id = 0
    id_map = {}
    for u in np.unique(a[:,:2]):
        id_map[u] = next_id
        next_id += 1
        a[:,:2][a[:,:2] == u] = id_map[u]
    return id_map

def load_netscience():
    import numpy as np
    import networkx as nx
    a = np.loadtxt('netscience.net', dtype=np.int)
    a = a[:, :2]
    make_ids_contiguous(a)
    return nx.from_edgelist(a)

def load_g(netname):
    import networkx as nx
    import numpy as np
    if netname == 'netscience':
        g = load_netscience()
    elif netname == 'homeless-a':
        G = load('a1.txt', directed=False)
        g = nx.from_numpy_matrix(G)
    elif netname == 'homeless-b':
        G = load('b1.txt', directed=False)
        g = nx.from_numpy_matrix(G)
    elif 'india' in netname:
        num = netname.split('-')[1]
        G = np.loadtxt('relations/' + num + '-All2.csv', delimiter=',')
        g = nx.from_numpy_matrix(G)
    elif netname == 'genrel':
        g = load_edgelist('genrel.txt')
    elif netname == 'SBM-1500-1':
        g = nx.read_edgelist('SBM_1500_1_1', nodetype=int)
    elif netname == 'SBM-equal_1000_0':
        g = nx.read_adjlist('SBM-equal_1000_0', nodetype=int)
    elif netname == 'SBM-unequal_1000_0':
        g = nx.read_adjlist('SBM-unequal_1000_0', nodetype=int)
    elif netname == 'residence':
        g = nx.read_edgelist('residence.txt', nodetype=int)
    return g

#Draw degree histogram with matplotlib.

def degree_distribution(fname, list_of_nodes, isolates):
    import collections
    import matplotlib.pyplot as plt
    import networkx as nx

    G = load_edgelist(fname)
    r = {}
    G.add_nodes_from(isolates)

    for x in list_of_nodes:
        r[x] = G.degree()[x]

    degree_sequence=sorted([d for d in r.values()], reverse=True) # degree sequence

    degreeCount=collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')

    plt.title("Degree Histogram")
    plt.ylabel("Count")
    plt.xlabel("Degree")
    
    plt.show()

# how many PLs are they connected to on average? 
def degree_withPL(fname, list_of_interest, pl_fname, isolates):
    import collections
    import matplotlib.pyplot as plt
    import networkx as nx

    G = load_edgelist(fname)
    r = {}
    G.add_nodes_from(isolates)

    pl_list = strip(pl_fname)

    for node in list_of_interest:
        a = [x for x in G.neighbors(node) if x in pl_list]
        r[node] = len(a)

    degree_sequence=sorted([d for d in r.values()], reverse=True) # degree sequence

    degreeCount=collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())

    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')

    plt.title("Degree Histogram")
    plt.ylabel("Count")
    plt.xlabel("Degree")
    #print(degree_sequence)
    plt.show()

#makes dictionary of shortest path lengths keyed by PLs
#each value is itself another dict, keyed by node number with value path length
def social_distance(g, list_of_interest):
    import networkx as nx
    d={}

    for node in list_of_interest:
        d[node] = nx.shortest_path_length(g,source=node)
    return d

def distance_from_PL(fname, pl_fname, convert_fname, nc_fname, isolates_fname):
    import networkx as nx
    import collections
    import matplotlib.pyplot as plt

    g = load_edgelist(fname)
    pl = strip(pl_fname)
    convert = strip(convert_fname)
    nc = strip(nc_fname)
    isolates = strip(isolates_fname)
    behavioral = convert + nc
    behavioral_without = [x for x in behavioral if x not in isolates]
    r = {} #large dictionary, keys are behavioral
    print(len(isolates), "length")
    d = social_distance(g, behavioral_without)
    for node in behavioral:
        if node in isolates:
            r[node] = 0
        else:
            p = {} #within dictionary, keys are pl, vals are distance, only take minimum in the end
            for peer_leader in pl:
                try:
                    p[peer_leader] = d[node][peer_leader]
                except KeyError:
                    print(peer_leader, node)
                    p[peer_leader]= 5
            min_distance = min(p.values())
            r[node] = min_distance
    print(r)

    for node in r.keys():
        if r[node]==5:
            r[node]=0
    c={}
    for each in convert:
        c[each] = r[each]

    distance_convert = sorted([a for a in c.values()], reverse=True)
    distance_sequence = sorted([a for a in r.values()], reverse=True) # degree sequence

    distanceCount=collections.Counter(distance_sequence)
    distanceConvert = collections.Counter(distance_convert)
    dis_c, cnt_c = zip(*distanceConvert.items())
    dis, cnt = zip(*distanceCount.items())
    print(distance_convert, len(distance_convert))
    print(distance_sequence, len(distance_sequence))

    plt.figure(1)
    plt.subplot(211)
    plt.bar(dis, cnt, width=0.80, color='b')

    plt.title("Distance from PL Histogram")
    plt.ylabel("Count")
    plt.xlabel("Distance")

    plt.subplot(212)
    plt.bar(dis_c, cnt_c, width=0.80, color = 'r')

    plt.ylabel("Count")
    plt.xlabel("Distance")
    # plt.show()


def visualize_set_separate(ICM, p, g, S, converted_list, total, fname, pl_fname, convert_fname, nc_fname, all_nodes):
    import networkx as nx
    import matplotlib.pyplot as plt

    pos=nx.spring_layout(g)
    total = [x for x in total if x not in S]
    converts = [x for x in converted_list if x in total]
    converts = [x for x in converts if x not in S]
    missing_converts = [x for x in converted_list if x not in total]
    missing_converts = [x for x in missing_converts if x not in S]
    
    n = [x for x in g.nodes() if x not in converted_list]
    missing_n = [x for x in n if x not in total]
    still_n = [x for x in n if x in total]

    all_n = still_n + missing_n
    n_colors = ['#565757']*len(still_n) + ['#E0E4E5']*len(missing_n)
    all_converts = converts + missing_converts
    converts_colors = ['#00C8FF']*len(converts) + ['#C1F0FE']*len(missing_converts)
  
    print('converts',all_converts)
    print('not', all_n)

    plt.figure(1)
    plt.subplot(211)
    nx.draw_networkx_labels(g, pos=pos,with_labels=True,font_size=8)

    nx.draw_networkx_nodes(g, pos= pos, nodelist= S, node_color = 'b', node_size=300)
    nx.draw_networkx_nodes(g, pos=pos, nodelist=all_converts, node_color = converts_colors, node_size=200)
    nx.draw_networkx_nodes(g, pos=pos, nodelist=all_n, node_color = n_colors, node_size = 200)

    # edges with missing nodes
    missing_edges = g.edges(missing_n + missing_converts)
    edge_colors = ['#E3E3E3'] * len(missing_edges)
    nx.draw_networkx_edges(g, pos=pos, width = 1.0)
    nx.draw_networkx_edges(g, pos=pos, edgelist= missing_edges, width = 0.5, edge_color = edge_colors)
    if ICM == True:
        plt.title('Simulated ICM with p= %.2f' % p)
    else:
        plt.title('Simulated TLM')

    plt.axis('off')

    visualize_set_actual(fname, pl_fname, convert_fname, nc_fname, all_nodes, pos)


def visualize_set(g, S, all_nodes):
    import networkx as nx
    node_color = []
    node_size = []
#    g = nx.subgraph(g, all_nodes)
    for v in g.nodes():
        if v in S:
            node_color.append('b')
            node_size.append(300)
        elif v in all_nodes:
            node_color.append('y')
            node_size.append(100)
        else:
            node_color.append('k')
            node_size.append(20)
#    node_size = [300 if v in S else 20 for v in g.nodes()]

    nx.draw(g, with_labels=True, node_color = node_color, node_size=node_size)


def visualize_set2(g, S, direct, convert):
    import networkx as nx

    node_color = []
    node_size = []
#    g = nx.subgraph(g, all_nodes)
    for v in g.nodes():
        if v in S:
            node_color.append('blue')
            node_size.append(300)  
        elif v in direct:
            if v in convert:
                node_color.append('green')
                node_size.append(200)
            else:
                node_color.append('yellowgreen')
                node_size.append(200)
        elif v in convert:
            node_color.append('red')
            node_size.append(200)
        else:
            node_color.append('k')
            node_size.append(20)
#    node_size = [300 if v in S else 20 for v in g.nodes()]
    nx.draw(g, with_labels=True, node_color = node_color, node_size=node_size)

def visualize_set_actual(fname, pl_fname, convert_fname, nc_fname, all_nodes, pos, min_color, max_color):
    import networkx as nx
    import matplotlib.pyplot as plt

    g = load_edgelist(fname)
    pl = strip(pl_fname)
    convert = strip(convert_fname)
    nc = strip(nc_fname)
    behavioral = convert + nc

    isolates = [x for x in all_nodes if x not in g.nodes()]
    print(isolates)
    g.add_nodes_from(isolates)
    # pos=nx.spring_layout(g)

    n = [x for x in all_nodes if x in nc]
    converts = [x for x in all_nodes if x in convert]
    missing = [x for x in all_nodes if x not in behavioral]
    missing = [x for x in missing if x not in pl]
  

    full = converts + n
    full_colors = [max_color]*len(converts)+[min_color+0.5]*len(n)
    n_colors = [min_color]*len(n)
    converts_colors = [max_color]*len(converts)
    missing_colors = ['#F0F2F4']*len(missing)
    print("v_min", min_color)
    print("v_max", max_color)

    blah = [max_color] * len(pl)
    # plt.subplot(311)
    plt.figure(1)
    nx.draw_networkx_labels(g,pos=pos,with_labels=True,font_size=8)

    nx.draw_networkx_nodes(g, pos= pos, nodelist= pl, node_color = 'r', node_size=300)
    nx.draw_networkx_nodes(g, pos=pos, nodelist=full, node_color = full_colors, cmap= plt.cm.PiYG,node_size=200)
    nx.draw_networkx_nodes(g, pos=pos, nodelist=missing, node_color = missing_colors, node_size = 200)

    # edges with missing nodes
    missing_edges = g.edges(missing)
    edge_colors = ['#E3E3E3'] * len(missing_edges)
    nx.draw_networkx_edges(g, pos=pos, width = 1.0)
    nx.draw_networkx_edges(g, pos=pos, edgelist= missing_edges, width = 0.5, edge_color = edge_colors)
    
    plt.title('Actual Network')
    plt.axis('off')
# sum up the neighborhood overlap for the entire PL set
# for each node, sum of overlap with every other PL nodee
def overlap(fname,pl_fname):
    import networkx as nx

    g = load_edgelist(fname)
    list_of_nodes = strip(pl_fname)
    list_of_nodes_2 = list_of_nodes[1:]
    overlap = []
    total = 0

    for node in list_of_nodes:
        for next in list_of_nodes_2:
            neighborhood = g.neighbors(next)
            neighborhood.append(next)
            overlap  = [x for x in g.neighbors(node) if x in neighborhood]
            print(overlap)
            node_neighbor = g.neighbors(node)
            node_neighbor.append(node)
            overlap = len(overlap)/float(len(node_neighbor+neighborhood))
            total = total + overlap
        if list_of_nodes_2: 
            list_of_nodes_2.pop(0)

    return total
            

def overlap2(fname,pl_list):
    import networkx as nx

    g = load_edgelist(fname)
    pl_list2 = pl_list[1:]
    overlap = []
    total = 0

    for node in pl_list:
        for next in pl_list2:
            neighborhood = g.neighbors(next)
            neighborhood.append(next)
            overlap  = [x for x in g.neighbors(node) if x in neighborhood]
            print(overlap)
            node_neighbor = g.neighbors(node)
            node_neighbor.append(node)
            overlap = len(overlap)/float(len(node_neighbor+neighborhood))
            total = total + overlap
        if pl_list2: 
            pl_list2.pop(0)

    return total

def overlap_onlyPL(fname,pl_list):
    import networkx as nx

    g = load_edgelist(fname)
    total = 0
    overlap = []

    for node in pl_list:
        overlap = [x for x in g.neighbors(node) if x in pl_list]
        print(overlap)
        overlap = len(overlap)/float(len(g.neighbors(node)))
        print(overlap)
        total = total + overlap
    return total

def redundancy(fname,pl_fname):
    import networkx as nx

    g= load_edgelist(fname)
    pl = strip(pl_fname)
    r = []
    penalty = []
    for each in pl:
        remove_this = [] 
        print("pl in question", each)
        ego = nx.ego_graph(g, each, radius = 1)
        n = len(ego.nodes())-1
        t = nx.edges(ego)
        print("edges", t)
        for node1, node2 in t:
            print("check", node1, node2)
            if node1 == each or node2 == each:
                remove_this.append((node1, node2))
                print("removed", node1, node2, each)
        print("removed this", remove_this)
        t = [x for x in t if x not in remove_this]     
        print("t", len(t))
        print("n", n)
        r.append(n-2*len(t)/float(n))
        penalty.append(2*len(t)/float(n))

    return penalty, sum(penalty), sum(r)


def percent_withPL(fname,pl_list):
    import networkx as nx

    g = load_edgelist(fname)
    total = 0
    overlap = []
    pl_list2  = pl_list[1:]
    total_degree = 0

    for node in pl_list:
        overlap = [x for x in g.neighbors(node) if x in pl_list2]
        overlap = len(overlap)
        # overlap = len(overlap)/float(len(g.neighbors(node)))
        print(overlap)
        skip = [x for x in pl_list if x not in pl_list2]
        print(skip)
        degree = len([x for x in g.neighbors(node) if x not in skip])
        print(degree)
        total_degree = total_degree + degree
        total = total + overlap
        if pl_list2: pl_list2.pop(0)

    
    return total/float(total_degree)

def spheres(fname, pl_fname, convert_fname, nc_fname):
    import networkx as nx

    g = load_edgelist(fname)
    pl_list = strip(pl_fname)
    convert = strip(convert_fname)
    nc = strip(nc_fname)
    total = nc+convert
    pl_list2 = pl_list[:]
    number_converted=0
    c={}
    n={}

    for node in pl_list:
        c[node] = [x for x in pl_list2 if x in g.neighbors(node)]
        n[node] = [x for x in pl_list2 if x not in g.neighbors(node)]

    s_c = {} 
    s_n = {}

    for key in c.keys():
        list_of_percent = []
        list_of_percent2 = []
        converted_original = [x for x in g.neighbors(key) if x in convert]
        only_original = [x for x in g.neighbors(key) if x not in pl_list]
        only_original = [x for x in only_original if x in total]

        by_itself_convert = len(converted_original)/float(len(only_original))
        list_of_percent.append(by_itself_convert)
        list_of_percent2.append(by_itself_convert)
        for each in c[key]:
            converted = [x for x in g.neighbors(each) if x in convert]
            converted = [x for x in converted if x not in pl_list]
            top = len(set(converted+converted_original))
            only = [x for x in g.neighbors(each) if x not in pl_list]
            only = [x for x in only if x in total]
            denom = len(set(only+only_original))
            if denom ==0:
                number_converted = 'denom 0'
            else:
                number_converted = top/float(denom)
            list_of_percent.append((each,number_converted, 'out of = %d' % denom))
        s_c[key] = list_of_percent
        for each in n[key]:
            if each == key:
                continue
            converted2 = [x for x in g.neighbors(each) if x in convert]
            converted2 = [x for x in converted2 if x not in pl_list]
            top = len(set(converted2 + converted_original))
            only = [x for x in g.neighbors(each) if x not in pl_list]
            only = [x for x in only if x in total]
            denom = len(set(only+only_original))
            if denom == 0:
                number_converted2 = 'denom 0'
            else:
                number_converted2 = top/float(denom)
            try:
                path = nx.shortest_path_length(g,source=key,target=each)
            except nx.exception.NetworkXNoPath:
                path = 'None'
            list_of_percent2.append((each, path,number_converted2, 'out of = %d' % denom))
        s_n[key] = list_of_percent2

    print(s_c)
    print(s_n)


def spheres_intersect(fname,pl_fname,convert_fname,nc_fname):
    import networkx as nx

    g = load_edgelist(fname)
    pl = strip(pl_fname)
    convert = strip(convert_fname)
    nc = strip(nc_fname)
    total = convert + nc
    only_neighbors_e = []
    only_neighbors_c = []
    counter_edge = 0
    counter_both = 0

    for each in total:
        try:
            pl_neighbors = [x for x in g.neighbors(each) if x in pl]
        except nx.exception.NetworkXError:
            continue
        number_pl_neighbors = len(pl_neighbors)
        if each in convert: 
            converted = 1
        else:
            converted = 0
        edge = 'no edge'
        if len(pl_neighbors):
            edge = len(pl_neighbors)
            counter_edge += 1
            if each in convert:
                counter_both += 1
            only_neighbors_e.append(edge)
            only_neighbors_c.append(converted)
    baseline_percent = len(convert)/float(len(total))
    edge_percent = counter_both/float(counter_edge)

    print(baseline_percent)
    print(edge_percent)
    print(only_neighbors_e)
    print(only_neighbors_c)

# returns list of all neighbors of a set of nodes 
def neighbors(g, S):
    import networkx as nx

    set_of_nbs = set()
    for node in S:
        for nb in g.neighbors(node):
            set_of_nbs.add(nb)
    return list(set_of_nbs)

def only_behavioral(direct, convert, nc):
    import networkx as nx

    behavioral = []
    for v in direct:
        if (v in convert) or (v in nc):
            behavioral.append(v)
    return behavioral

def visualize_communities(fname, convert_fname, nc_fname, pl_fname):
    import networkx as nx
    import community
    import numpy as np
    import random
    import matplotlib.pyplot as plt

    g = load_edgelist(fname)
    convert = strip(convert_fname)
    nc = strip(nc_fname)
    S = strip(pl_fname)

    total = convert + nc

    isolates = [x for x in total if x not in g.nodes()]
    g.add_nodes_from(isolates)

    part = community.best_partition(g)
    copy_part = part
    print(part)
    part = [part[x] for x in g.nodes()]
    print(part)
    com_names = np.unique(part)
    communities = []
    node_contig = range(len(g.nodes()))


    for i,c in enumerate(com_names):
        print(com_names)
        communities.append([])
        print(communities, i)
        print(communities[0], "communities 0")
        print(communities)
        test = [x for x in node_contig if part[x]==c]
        print(test)
        communities[i].extend([x for x in node_contig if part[x] == c])
    node_color = part
    pos = nx.layout.spring_layout(g, k=0.1)

    labels={}
    for node in S:
        labels[node] = node

    nx.draw(g, with_labels=False, node_color=node_color, pos=pos, node_size=50)
    nx.draw_networkx_labels(g,pos,labels,font_size=12,font_color='black')

    print(part)
    print(copy_part)

    plt.show()

def write_for_metis(g, fname):
    with open(fname, 'w') as f:
        f.write(str(g.number_of_nodes()) + ' ' + str(g.number_of_edges()) + '\n')
        for v in g.nodes():
            for u in g.neighbors(v):
                f.write(str(u+1) + ' ')
            f.write('\n')
  
def recursive_split(g):
    if len(g) == 1:
        return [g.nodes()[0]]
    return [recursive_split(g.subgraph(g.nodes()[:len(g)/2])), recursive_split(g.subgraph(g.nodes()[len(g)/2:]))]    
      
def recursive_partition(g, fname):
    print('CALL', len(g))
    import subprocess
    import numpy as np
    import networkx as nx
    if len(g) == 1:
        return [g.nodes()[0]]
    if g.number_of_edges() == 0:
        return recursive_split(g)

    contig_g = nx.from_numpy_matrix(nx.to_numpy_matrix(g))
    write_for_metis(contig_g, fname)
    subprocess.call(['rm', fname + '.part.2'])
    subprocess.call(['./gpmetis', fname, '2'])
    labels = np.loadtxt(fname + '.part.2')
    nodes = np.array(g.nodes())
    print(len(nodes), len(labels))
    part1 = nodes[labels == 0]
    part2 = nodes[labels == 1]
    if len(part1) == 0:
        return recursive_split(g.subgraph(part2))      
    if len(part2) == 0:
        return recursive_split(g.subgraph(part1))
    return [recursive_partition(g.subgraph(part1), fname), recursive_partition(g.subgraph(part2), fname)]

def greedy_icm(g, budget, rr_sets = None, start_S = None):
    from rr_icm import make_rr_sets_cython, eval_node_rr
    import heapq
    num_nodes = len(g)
    allowed_nodes = range(num_nodes)
    if rr_sets == None:
        rr_sets = make_rr_sets_cython(g, 500, range(num_nodes))
    if start_S == None:
        S = set()
    else:
        S = start_S
    upper_bounds = [(-eval_node_rr(u, S, num_nodes, rr_sets), u) for u in allowed_nodes]    
    heapq.heapify(upper_bounds)
    starting_objective = 0
    #greedy selection of K nodes
    while len(S) < budget:
        val, u = heapq.heappop(upper_bounds)
        new_total = eval_node_rr(u, S, num_nodes, rr_sets)
        new_val =  new_total - starting_objective
        #lazy evaluation of marginal gains: just check if beats the next highest upper bound
        if new_val >= -upper_bounds[0][0] - 0.1:
            S.add(u)
            starting_objective = new_total
        else:
            heapq.heappush(upper_bounds, (-new_val, u))
    return S, starting_objective

def greedy(items, budget, f):
    '''
    Generic greedy algorithm to select budget number of items to maximize f.
    
    Employs lazy evaluation of marginal gains, which is only correct when f is submodular.
    '''
    import heapq
    upper_bounds = [(-f(set([u])), u) for u in items]    
    heapq.heapify(upper_bounds)
    starting_objective = f(set())
    S  = set()
    #greedy selection of K nodes
    while len(S) < budget:
        val, u = heapq.heappop(upper_bounds)
        new_total = f(S.union(set([u])))
        new_val =  new_total - starting_objective
        #lazy evaluation of marginal gains: just check if beats the next highest upper bound
        if new_val >= -upper_bounds[0][0] - 0.1:
            S.add(u)
            starting_objective = new_total
        else:
            heapq.heappush(upper_bounds, (-new_val, u))
    return S, starting_objective

In [3]:
############ MAIN FUNCTIONS FOR RUNNING THE LINEAR THRESHOLD MODEL ############

#This function simulates how diffusion occurs according to the LTM, a standard information diffusion model
def simulate_LTM(g, pl, time_limit):
	converted_list = pl[:]
	threshold = {}

	for node in g.nodes():
		threshold[node] = random()
	for t in range(time_limit):
		converted_list1 = converted_list[:]
		for node in g.nodes():
			total_weight = 0
			if g.degree()[node]:
				weight = 1/float(g.degree()[node])
			else:
				continue
			for each in g.neighbors(node):
				if each in converted_list:
					total_weight = total_weight + weight
			if total_weight > threshold[node]:
				converted_list.append(node)
		if set(converted_list1) == set(converted_list):
			#print('set', t)
			break
	return converted_list #This returns all the nodes in the network that have been activated/converted in the diffusion process

#This function runs the LTM with the necessary parameters
def main(g, all_nodes, pl, convert, nc, time_limit, number): 
    r = {}
    new_range = np.linspace(0,1,100).tolist()
    
    all_nodes1 = []
    for sublist in all_nodes:
        all_nodes1.append(sublist[0])
        
    all_nodes_except_PL = [x for x in all_nodes1 if x not in pl]
	
    for node in all_nodes_except_PL:
        r[node] = 0

    #Calculate the False Positive Rates and True Positive Rates by comparing predicted vs observed node activations
    isolates = [x for x in all_nodes1 if x not in g.nodes()]
    g.add_nodes_from(isolates)
    binary_index = []
    for converted in all_nodes_except_PL:
        if converted in convert:
            binary_index.append(1)
        elif converted in nc:
            binary_index.append(0)
        else:
            binary_index.append(-1)
    FPR_total = []
    TPR_total = []

    sensitivity, specificity = 0, 0
    running_s = []
    running_p = []
    			
    running_binary_sim = [0]*len(all_nodes_except_PL)
    number_of_predicted_positives = np.zeros(number)
    
    for i in range(number):
        converted_list = simulate_LTM(g, pl, time_limit)
        converted_list = [x for x in converted_list if x not in pl]
        binary_sim = []
        for each in all_nodes_except_PL:
            if each in converted_list:
                binary_sim.append(1)
            else:
                binary_sim.append(0)
        running_binary_sim = [sum(x) for x in zip(binary_sim, running_binary_sim)]
        number_of_predicted_positives[i] = sum(binary_sim)
      
    running_binary_sim = [x/float(number) for x in running_binary_sim]
    
    running_binary_sim = np.array(running_binary_sim)
    binary_index = np.array(binary_index)
    have_data = np.where(binary_index != -1)[0]
    
    max_auc_sklearn = sklearn.metrics.roc_auc_score(binary_index[have_data], running_binary_sim[have_data])
    roc_curve = sklearn.metrics.roc_curve(binary_index[have_data], running_binary_sim[have_data])
    
    avg_predicted_positives = np.round(np.mean(number_of_predicted_positives))
    observed_positives = np.size(np.where(binary_index == 1))
        
    print('AUC:', max_auc_sklearn)
    
    return max_auc_sklearn, avg_predicted_positives, observed_positives, roc_curve

#This function runs 'main' function multiple times to derive average performance scores
def multiple_runs(g, all_nodes, pl, convert, nc, time_limit, number, number_of_simulations):
    
    running_auc_results = []
    avg_predicted_positives_list = []
    observed_positives_list = []
    ratio_predicted_to_observed_positives_list = []
    running_fpr_results = []
    running_tpr_results = []
    
    for i in range (number_of_simulations):
        max_auc_sklearn, avg_predicted_positives, observed_positives, roc_curve = main(g, all_nodes, pl, convert, nc, time_limit, number)
        
        ratio_predicted_to_observed_positives = avg_predicted_positives/observed_positives
        
        print('Ratio of predicted to observed positives:', ratio_predicted_to_observed_positives)
        
        running_auc_results.append(max_auc_sklearn)
        avg_predicted_positives_list.append(avg_predicted_positives)
        observed_positives_list.append(observed_positives)
        ratio_predicted_to_observed_positives_list.append(ratio_predicted_to_observed_positives)
        running_fpr_results.append(roc_curve[0])
        running_tpr_results.append(roc_curve[1])
        
        print("*****", i+1, " simulations run *****")

    auc_mean = np.mean(running_auc_results)
    auc_stdev = np.std(running_auc_results)
    avg_predicted_positives_mean = np.mean(avg_predicted_positives_list)
    observed_positives_mean = np.mean(observed_positives_list)
    ratio_predicted_to_observed_positives_mean = np.mean(ratio_predicted_to_observed_positives_list)
    ratio_predicted_to_observed_positives_std = np.std(ratio_predicted_to_observed_positives_list)

    #Find macro-average of ROC curves
    mean_fpr = np.unique(np.concatenate([running_fpr_results[i] for i in range(number_of_simulations)])) #Aggregate all false positive rates
    mean_tpr = np.zeros_like(mean_fpr) #Interpolate all ROC curves at these points
    
    temporary_tpr = np.zeros_like(mean_fpr) 
    temporary_count = np.zeros_like(mean_fpr)
    
    for k in range(np.size(mean_fpr)):
        fpr_value = mean_fpr[k]
        for i in range(number_of_simulations):
            temporary_tpr[k] += np.sum(running_tpr_results[i][np.where(running_fpr_results[i] == fpr_value)])
            temporary_count[k] += np.size(np.where(running_fpr_results[i] == fpr_value))
    mean_tpr = temporary_tpr/temporary_count

   
    return auc_mean, auc_stdev, ratio_predicted_to_observed_positives_mean, ratio_predicted_to_observed_positives_std, avg_predicted_positives_mean, observed_positives_mean, running_fpr_results, running_tpr_results, mean_fpr, mean_tpr



In [4]:
###### ACTUALLY RUN THE MODEL ########
#File directories
fname = 'xxxx/MFP_GSN_undirected_trimmed_Baseline.txt'
pl_fname = 'xxxx/MFP_PL.txt'
convert_fname = 'xxxx/MFP_ConfirmedConversations_3M.txt'
nc_fname = 'xxxx/MFP_nc_1M.txt'
all_nodes_fname = 'xxx/MFP_GSN_untrimmed_allnodes.txt'

#Descriptions of files
#'fname' is a list of the node IDs of all members in your social network
#'pl_fname' is a list of the node IDs of the Peer Leaders in your network
#'convert_fname' is a list of the node IDs of the members who were observed to be converted
#'nc_fname' is a list of the node IDs of the members who were NOT observed to be converted
#'all_nodes_fname' is a complete list of all named and unnamed node IDs in the network

#Load edge lists etc.
g = load_edgelist(fname) #Load up edge list as a graph edge list
pl = strip(pl_fname) #List of peer leaders
convert = strip(convert_fname)
nc = strip(nc_fname)
all_nodes = strip_demo(all_nodes_fname)

#Parameters to change in the model:
time_limit = 100 #Time limit after which the simulation ends (usually set at 100)
number = 100 # Number of times each simulation is iterated (usually set at 100)
number_of_simulations = 10 #Number of separate simulations you want to average over (usually set at 10)

#This line executes the interrelated functions to produce performance metrics of the LTM simulations
auc_mean, auc_stdev, ratio_predicted_to_observed_positives_mean, ratio_predicted_to_observed_positives_std, avg_predicted_positives_mean, observed_positives_mean, running_fpr_results, running_tpr_results, mean_fpr, mean_tpr = multiple_runs(g, all_nodes, pl, convert, nc, 100, 100, 10)

print(auc_mean)

AUC: 0.153153153153
Ratio of predicted to observed positives: 0.540540540541
***** 1  simulations run *****
AUC: 0.117117117117
Ratio of predicted to observed positives: 0.567567567568
***** 2  simulations run *****
AUC: 0.121621621622
Ratio of predicted to observed positives: 0.567567567568
***** 3  simulations run *****
AUC: 0.126126126126
Ratio of predicted to observed positives: 0.567567567568
***** 4  simulations run *****
AUC: 0.198198198198
Ratio of predicted to observed positives: 0.540540540541
***** 5  simulations run *****
AUC: 0.193693693694
Ratio of predicted to observed positives: 0.567567567568
***** 6  simulations run *****
AUC: 0.153153153153
Ratio of predicted to observed positives: 0.567567567568
***** 7  simulations run *****
AUC: 0.117117117117
Ratio of predicted to observed positives: 0.567567567568
***** 8  simulations run *****
AUC: 0.175675675676
Ratio of predicted to observed positives: 0.594594594595
***** 9  simulations run *****
AUC: 0.162162162162
Ratio of